# DDB API: `institutions`

OpenAPI-Dokumentation: https://labs.deutsche-digitale-bibliothek.de/app/ddbapi/#/institutions

In [ ]:
# Modulimport
import requests # Standardbibliothek zur Interaktion mit Servern über HTTP
from matplotlib import pyplot as plt
from collections import Counter

In [ ]:
# APIKEY wird entweder aus einer Datei eingelesen
try:
    with open('../apikey.txt', 'r') as IN:
        APIKEY = IN.read()
# oder manuell eingegeben
except:
    APIKEY = ""

In [ ]:
APIKEY

In [ ]:
endpunkt = "https://api.deutsche-digitale-bibliothek.de/institutions"

# HTTP Request-Header
headers = {
    'Accept' : 'application/json',
    # Authentifizierung auch über Header möglich:
    'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

# Parameter, die an den Endpunkt angefügt werden
parameters = {
    'hasItems' : "false",
    #'sector' : "sec_01", # optional, mögliche Werte: sec_01 bis sec_07. Kein Wert heißt alle Sektoren
    # Authentifizierung über Paramter:
    #"oauth_consumer_key" : APIKEY
}

res = requests.get(endpunkt, headers = headers, params = parameters)

Die Antwort ist im JSON-Format:

In [ ]:
res.json()[:2]

Man kann sich also eine Liste der Institutionen ausgeben lassen (hier die ersten 5):

In [ ]:
for institution in res.json()[:5]:
    print(institution['id'], institution['name'], institution['sector'])

Man kann auch schon erste Statistiken erheben. Wie verteilen sich die Insitutionen auf die verschiedenen Sektoren?

In [ ]:
Counter([institution['sector'] for institution in res.json()])

Diese Zahlen lassen sich auch visualisieren:

In [ ]:
plt.title("Verteilung der GLAMs auf DDB nach Sektoren")
plt.hist(sorted([institution['sector'] for institution in res.json()]))
plt.show()

### Wie viele Institutionen liefern Daten an die DDB?

In [ ]:
endpunkt = "https://api.deutsche-digitale-bibliothek.de/institutions"

# HTTP Request-Header
headers = {
    'Accept' : 'application/json',
    # Authentifizierung auch über Header möglich:
    'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

# Parameter, die an den Endpunkt angefügt werden
parameters = {
    #'sector' : "sec_01", # optional, mögliche Werte: sec_01 bis sec_07. Kein Wert heißt alle Sektoren
    # Authentifizierung über Paramter:
    #"oauth_consumer_key" : APIKEY
}

In [ ]:
parameters['hasItems'] = "false" # zeige alle Institutionen

institutionenGesamt = len(requests.get(endpunkt, headers = headers, params = parameters).json())

parameters['hasItems'] = "true" # zeige nur Institutionen mit digitalen Objekten

institutionenMitObjekten = len(requests.get(endpunkt, headers = headers, params = parameters).json())
institutionenOhneObjekte = institutionenGesamt - institutionenMitObjekten

In [ ]:
plt.title("Wie viele Institutionen liefern Digitalisate an die DDB?")
plt.bar(["mit Digitalisaten", "ohne Digitalisate"], [institutionenMitObjekten, institutionenOhneObjekte])
plt.show()

### Wie viele Institutionen liefern Daten an die DDB je Sparte?

Hier lohnt es sich jetzt, das oben verwendete Vorgehen in eine Funktion zu packen.

In [ ]:
# Liste der Sektoren
headers = {
    'Accept' : 'application/json',
    'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
}

res = requests.get('https://api.deutsche-digitale-bibliothek.de/institutions/sectors', headers = headers)

sectorList = [_['value'] for _ in res.json()]

In [ ]:
def institutionJeSparte(sparte):
    
    endpunkt = "https://api.deutsche-digitale-bibliothek.de/institutions"

    # HTTP Request-Header
    headers = {
        'Accept' : 'application/json',
        'Authorization' : f'OAuth oauth_consumer_key="{APIKEY}"'
    }

    # Parameter, die an den Endpunkt angefügt werden
    parameters = {
        'sector' : sparte
    }
    
    # mit Objekten
    parameters['hasItems'] = "true"
    mitObjekten = len(requests.get(endpunkt, headers = headers, params = parameters).json())
    
    # ohne Objekte
    parameters['hasItems'] = "false"
    gesamt = len(requests.get(endpunkt, headers = headers, params = parameters).json())
    ohneObjekte = gesamt - mitObjekten
    
    return (mitObjekten, ohneObjekte)

In [ ]:
mitObjektenListe = list()
ohneObjekteListe = list()

for s in sectorList:
    mit,ohne = institutionJeSparte(s)
    mitObjektenListe.append(mit)
    ohneObjekteListe.append(ohne)

In [ ]:
fix, ax = plt.subplots(layout='constrained')
balkenBreite = 0.4

for n, sec in enumerate(sectorList):
    rects = ax.bar([n, n + balkenBreite], [ mitObjektenListe[n], ohneObjekteListe[n] ], balkenBreite, color = ["green", "red"] )
    ax.bar_label(rects, padding = 3)
    
ax.set_ylabel('Zahl der Einrichtungen')
ax.set_title('Datenliefernde Einrichtungen je Sparte')
ax.set_xticks([_ + balkenBreite/2 for _ in range(len(objekteJeSparte))], objekteJeSparte.keys() )
plt.show()